# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [11]:
import os
import pandas as pd
import numpy as np

from sklearn.utils import resample
import joblib

from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [14]:
ws = Workspace.from_config()
experiment_name = 'credit_eval_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [15]:
def get_data():
    columns = ['A'+str(i) for i in range(1,17)]
    df = pd.read_csv('data.csv', header = None, names = columns )
    df.dropna(inplace = True)

    # Removing Null
    for column in df.columns:
        df = df[df[column] != '?']

    # Converting attributes to binary
    df['A16'] = df['A16'].map({'+': 1, '-':0})    
    df['A1']  = df['A1'].map({'b': 1, 'a':0})
    df['A9']  = df['A9'].map({'t': 1, 'f':0})
    df['A10'] = df['A10'].map({'t': 1, 'f':0})
    df['A12'] = df['A12'].map({'t': 0, 'f':1})
    df['A13'] = df['A13'].map({'g': 'mm', 'p':'kk'})

    # Conversting categorical data into onehot encoding
    cat_columns = ['A4', 'A5', 'A6', 'A7', 'A13']
    for column in cat_columns:
        dummies = pd.get_dummies(df[column])
        df[dummies.columns] = dummies
        df.drop(columns = column,inplace = True)
    df = df.astype(float)
    df['A16'] = df['A16'].astype(int)
    return df

In [16]:
dataset_name = 'credit_approval_data'

# Check Registry
if dataset_name in ws.datasets.keys(): 
    dataset = ws.datasets[dataset_name] 

    print(f'Located {dataset_name} in Registry.')

# Read from UCI ML Repository
else:
    df = get_data()
    
    # Register the dataset
    datastore = ws.get_default_datastore()

    dataset = Dataset.Tabular.register_pandas_dataframe(
        dataframe=df, 
        name=dataset_name, 
        description='A dataset of credit approved and rejected applications',
        target=datastore
    )
    

Located credit_approval_data in Registry.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [19]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
    "C": choice(0.01, 0.1, 0.1, 1, 10),
    "max_iter": choice(25, 50, 100, 200, 500),
    "solver": choice('lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga')
    }
    )

#TODO: Create your estimator and hyperdrive config
if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

compute_target = 'ml-compute'

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', 
                script='train.py', 
                compute_target=compute_target,
                environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
hyperparameter_sampling=param_sampling,
policy=early_termination_policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=15,
max_concurrent_runs=4
)

In [20]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_0a70fe1c-f103-448c-a09f-38e93a07eb89
Web View: https://ml.azure.com/runs/HD_0a70fe1c-f103-448c-a09f-38e93a07eb89?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-225216/workspaces/quick-starts-ws-225216&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-02-09T01:17:18.384182][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-02-09T01:17:19.5568947Z][SCHEDULER][INFO]Scheduling job, id='HD_0a70fe1c-f103-448c-a09f-38e93a07eb89_0' 
[2023-02-09T01:17:19.6313847Z][SCHEDULER][INFO]Scheduling job, id='HD_0a70fe1c-f103-448c-a09f-38e93a07eb89_1' 
[2023-02-09T01:17:19.7743938Z][SCHEDULER][INFO]Scheduling job, id='HD_0a70fe1c-f103-448c-a09f-38e93a07eb89_2' 
[2023-02-09T01:17:19.799109][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-02-09T01:17:19.8808212Z][SCHEDULER][INFO]Scheduling job, id='HD_0a70fe1c-f103-448c-a09f-38e93a07eb

{'runId': 'HD_0a70fe1c-f103-448c-a09f-38e93a07eb89',
 'target': 'ml-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-02-09T01:17:17.321495Z',
 'endTimeUtc': '2023-02-09T01:33:37.386611Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8ba0c73a-6a65-43c2-9645-29d057a99bc9',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': '125',
  'score': '0.8396946564885496',
  'best_child_run_id': 'HD_0a70fe1c-f103-448c-a09f-38e93a07eb89_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_0a70fe1c-f103-448c-a09f-38e93a07eb89_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientT

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [30]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Best Run
print("Best Run: ")
print("")

# Accuracy
print("Accuracy: " + str(best_run.get_metrics()['Accuracy']))
print("")

# Metrics
print("Metrics: " + str(best_run.get_metrics()))
print("")


# Tags
print("Tags: " + str(best_run.get_tags()))
print("")

Best Run: 

Accuracy: 0.8396946564885496

Metrics: {'Regularization Strength:': 0.1, 'Max iterations:': 200, 'Solver:': 'liblinear', 'Accuracy': 0.8396946564885496}

Tags: {'_aml_system_hyperparameters': '{"C": 0.1, "max_iter": 200, "solver": "liblinear"}', 'hyperparameters': '{"C": 0.1, "max_iter": 200, "solver": "liblinear"}', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}'}



In [31]:
best_run.register_model('hyperdrive-best-model.joblib', '.')

Model(workspace=Workspace.create(name='quick-starts-ws-225216', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-225216'), name=hyperdrive-best-model.joblib, id=hyperdrive-best-model.joblib:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

